In [1]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('whatever.vol')
mesh.Curve(2)
# mesh.Refine()

In [2]:
%run J.ipynb

using 3d mesh with ne= 21002  elements and nv= 3732  points
Solving...
... Success!
Number of iterations = 413/1000 | Residual = 9.922358609037997e-06
2999.999999999988
Solving...
... Success!
Number of iterations = 344/1000 | Residual = 9.320419900073156e-06
3000.003783169095


In [3]:
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"

BH data from TEAM 13 problem
returning energy


In [4]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

p = 2

HCurl = ngs.HCurl(mesh, order = p, nograds = True)
u,v = HCurl.TnT()


# Nonlinear:

maxit = 10000
tol2 = 1e-8
regb = 1e-12

A = ngs.GridFunction(HCurl)
B = ngs.curl(A)
normB = ngs.sqrt(B*B + regb)



cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B)
def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    return res



cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0)
# Draw(mu_nl, mesh)
# rhs = ngs.LinearForm(- (Hs*ngs.curl(v))*ngs.dx)
# rhs = ngs.LinearForm(- (ngs.curl(Hs)*v)*ngs.dx)
rhs = ngs.LinearForm((cf_rhs*ngs.curl(A)*ngs.curl(v) - Hs*ngs.curl(v))*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs



Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1], B[0]*B[2],
              B[1]*B[0], B[1]*B[1], B[1]*B[2],
              B[2]*B[0], B[2]*B[1], B[2]*B[2]), dims =(3,3))

fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id)

K_iter = ngs.BilinearForm(HCurl, nograds = True)
K_iter += (cf_iter*ngs.curl(u)*ngs.curl(v))*ngs.dx
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [5]:
print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,HCurl.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0,0,0)))

du = ngs.GridFunction(HCurl)
uo = ngs.GridFunction(HCurl)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(HCurl.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = HCurl.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, atol  = 1e-5,  maxiter = maxit)
    du.vec.data = iterativeSolver * dw.vec 
    
    if len(iterativeSolver.residuals) == maxit: print("... Success!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm

    if nrm/nrm0<tol2:
        print("converged to desired tolerance")
        break
    elif abs(wo-w) < tol2*1e-2:
        print("stopped early due to stagnation")
        break
    else:
        # linesearch
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.1*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.2e | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                A.vec.data = uo.vec.data

Using 3D mesh with ne= 21002 elements and nv= 3732 points and  109428 DOFs.
 
Iter:  1 | assem : 0.31s | CG took 0.73s with  218 iterations | alpha : 1.00 | energy = 4.28e-13 | relres = 1.00e+00 |
Iter:  2 | assem : 0.30s | CG took 1.03s with  304 iterations | alpha : 0.25 | energy = -1.24e+00 | relres = 8.59e-01 |
Iter:  3 | assem : 0.23s | CG took 0.68s with  201 iterations | alpha : 0.50 | energy = -1.59e+00 | relres = 2.19e-02 |
Iter:  4 | assem : 0.30s | CG took 0.59s with  173 iterations | alpha : 1.00 | energy = -1.60e+00 | relres = 3.23e-03 |
Iter:  5 | assem : 0.24s | CG took 0.40s with  119 iterations | alpha : 1.00 | energy = -1.61e+00 | relres = 1.52e-04 |
Iter:  6 | assem : 0.23s | CG took 0.26s with   76 iterations | alpha : 1.00 | energy = -1.61e+00 | relres = 6.97e-06 |
Iter:  7 | assem : 0.23s | CG took 0.15s with   43 iterations | alpha : 0.50 | energy = -1.61e+00 | relres = 3.08e-06 |
Iter:  8 | assem : 0.28s | CG took 0.09s with   25 iterations | alpha : 0.50 | ener

In [6]:
# Using 3D mesh with ne= 21002 elements and nv= 3732 points and  109428 DOFs.
 
# Iter:  1 | assem : 0.30s | CG took 0.75s with  218 iterations | alpha : 1.00 | energy = 4.28e-13 | relres = 1.00e+00 |
# Iter:  2 | assem : 0.30s | CG took 1.03s with  304 iterations | alpha : 0.25 | energy = -1.24e+00 | relres = 8.59e-01 |
# Iter:  3 | assem : 0.25s | CG took 0.68s with  201 iterations | alpha : 0.50 | energy = -1.59e+00 | relres = 2.19e-02 |
# Iter:  4 | assem : 0.32s | CG took 0.60s with  173 iterations | alpha : 1.00 | energy = -1.60e+00 | relres = 3.23e-03 |
# Iter:  5 | assem : 0.26s | CG took 0.40s with  119 iterations | alpha : 1.00 | energy = -1.61e+00 | relres = 1.52e-04 |
# Iter:  6 | assem : 0.25s | CG took 0.26s with   76 iterations | alpha : 1.00 | energy = -1.61e+00 | relres = 6.97e-06 |
# Iter:  7 | assem : 0.26s | CG took 0.15s with   43 iterations | alpha : 0.50 | energy = -1.61e+00 | relres = 3.08e-06 |
# Iter:  8 | assem : 0.31s | CG took 0.09s with   25 iterations | alpha : 0.50 | energy = -1.61e+00 | relres = 4.37e-07 |
# Iter:  9 | assem : 0.31s | CG took 0.07s with   21 iterations | alpha : 0.50 | energy = -1.61e+00 | relres = 8.18e-08 |
# Iter: 10 | assem : 0.25s | CG took 0.06s with   18 iterations | alpha : 0.50 | energy = -1.61e+00 | relres = 4.33e-08 |
# Iter: 11 | assem : 0.26s | CG took 0.05s with   16 iterations | alpha : 0.25 | energy = -1.61e+00 | relres = 2.42e-08 |
# converged to desired tolerance

In [7]:
L2 = ngs.VectorL2(mesh, order=2)
Bl2 = ngs.GridFunction(L2)
with ngs.TaskManager(): Bl2.Set(B)
Bl2np = Bl2.vec.FV().NumPy()
HCurl.ndof

import matplotlib.pyplot as plt
import numpy as np
max(abs(Bl2np))

1.7872970510694084

In [10]:
help(CGSolver)

Help on class CGSolver in module ngsolve.krylovspace:

class CGSolver(LinearSolver)
 |  CGSolver(*args, conjugate: bool = False, abstol: float = None, maxsteps: int = None, printing: bool = False, **kwargs)
 |  
 |  Method resolution order:
 |      CGSolver
 |      LinearSolver
 |      ngsolve.la.BaseMatrix
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, conjugate: bool = False, abstol: float = None, maxsteps: int = None, printing: bool = False, **kwargs)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: ngsolve.la.BaseMatrix) -> None
 |      
 |      2. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.la.BaseVector) -> None
 |      
 |      3. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.la.MultiVector) -> None
 |      
 |      4. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.bla.MatrixD) -> None
 |      
 |      5. __init__(self: ngsolve.la.B

In [8]:
help(ngs.comp.GridFunctionCoefficientFunction)

Help on class GridFunctionCoefficientFunction in module ngsolve.comp:

class GridFunctionCoefficientFunction(ngsolve.fem.CoefficientFunction)
 |  Method resolution order:
 |      GridFunctionCoefficientFunction
 |      ngsolve.fem.CoefficientFunction
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Trace(...)
 |      Trace(self: ngsolve.comp.GridFunctionCoefficientFunction) -> ngsolve.comp.GridFunctionCoefficientFunction
 |      
 |      take canonical boundary trace.
 |  
 |  __getstate__(...)
 |      __getstate__(self: ngsolve.comp.GridFunctionCoefficientFunction) -> tuple
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __setstate__(...)
 |      __setstate__(self: ngsolve.comp.GridFunctionCoefficientFunction, arg0: tuple) -> None
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  _

In [9]:
b = ngs.curl(gfu)
nrmb = sqrt(b*b+regb)
tmp = (1/2*mu0)*b*b
cf_w = CoefficientFunction([tmp, fun_w(nrmb), tmp, tmp])
cf_J = J*gfu
def funW():
    return Integrate(cf_w-cf_J, mesh)

cf_nut = CoefficientFunction([1/mu0, fun_dw(nrmb)/nrmb, 1/mu0, 1/mu0]) 
lf_dw = LinearForm(fes)
lf_dw += (cf_nut*b*curl2d(v) - J*v) * dx

def fundW():
    lf_dw.Assemble()
    return lf_dw


Id = CoefficientFunction((1,0,0,1), dims=(2,2))
bbt = CoefficientFunction((b[0]*b[0],b[0]*b[1],b[1]*b[0],b[1]*b[1]),dims=(2,2))
fun1 = fun_dw(nrmb)/nrmb
fun2 = (fun_ddw(nrmb) - fun_dw(nrmb)/nrmb)/(nrmb*nrmb)
# cf_nu = CoefficientFunction([1/mu0, fun_ddw(nrmb),1/mu0, 1/mu0])
# cf_nu = CoefficientFunction([1/mu0, fun1, 1/mu0, 1/mu0])
cf_nu = CoefficientFunction([1/mu0*Id, fun1*Id+fun2*bbt, 1/mu0*Id, 1/mu0*Id])
bf_da = BilinearForm(fes, symmetric=True)
bf_da += (cf_nu*curl2d(u)*curl2d(v))*dx
def funddW():
    bf_da.Assemble()
    return bf_da

# solve again with newton
Draw(b, mesh, "b")
Draw(cf_nut * b, mesh, "h")

gfu.Set(0)
# Redraw()

du = GridFunction(fes)
uo = GridFunction(fes)
wo = 1e12
tic()
for it in range(1,maxit+1):
    # with TaskManager():
    w = funW()
    dw = fundW()
    da = funddW()
    du.vec.data = da.mat.Inverse(fes.FreeDofs(), inverse="pardiso") * dw.vec 
    nrm = InnerProduct(du.vec,dw.vec)
    if it==1:
        nrm0=nrm

    print("it", it, ": w=",w, ", relres=", nrm/nrm0)

    if nrm/nrm0<tol2:
        print("converged to desired tolerance")
        break
    elif abs(wo-w)<tol2*1e-2:
        print("stopped early due to stagnation")
        break
    else:
        # linesearch
        uo.vec.data = gfu.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            gfu.vec.data -= alpha*du.vec.data
            wn = funW()
            if wn < w - alpha*0.1*nrm:
                print("alpha=",alpha)
                break
            else:
                alpha=alpha/2
                gfu.vec.data = uo.vec.data

    Redraw()
print("nonlinear solve required", toc(), "sec")

NameError: name 'gfu' is not defined

In [ ]:
p = 1

HCurl = ngs.HCurl(mesh, order = p)
u,v = HCurl.TnT()

# K = ngs.BilinearForm(HCurl, symmetric = True)
# K+= ngs.curl(u)*ngs.curl(v)*ngs.dx

K = ngs.BilinearForm(ngs.curl(u)*ngs.curl(v)*ngs.dx, symmetric = True)
Kpre = ngs.Preconditioner(K, type = "local")

f = ngs.LinearForm(ngs.curl(Hs) * v * ngs.dx)

with ngs.TaskManager():
    K.Assemble()
    f.Assemble()



print("starting nonlinear solver")

